In [ ]:
# Installing necessary libraries
import pandas as pd
import numpy as np
import os
import requests
import json
import datetime # for more modules later

In [ ]:
# Create a list with all files in the folder 
# change file path
    # find file path if not working
    # print(f"your current wd is: {os.getcwd()}")
folderpath = r"2022-citibike-tripdata"
filepath = [os.path.join(folderpath, name) for name in os.listdir(folderpath)]
filepath

In [ ]:
# had to use zipfile to open because read_csv can"t read multiple files from one zip
import zipfile

# pick important columns
columns = ["ride_id", "started_at", "ended_at", "start_station_name", "end_station_name"]

# create emtpy dataframe for citi bike data 
citi_df = pd.DataFrame()

# go through every zip in the filepath (12 for each month)
for zip_file in filepath:
    # Use zipfile to open each file
    zf = zipfile.ZipFile(zip_file)
    # read files individually
    dfs = [pd.read_csv(zf.open(f), usecols = columns) for f in zf.namelist()]
    # combine all files from singular zip file
    df = pd.concat(dfs,ignore_index=True)
    # add to citi_df
    citi_df = pd.concat([citi_df, df], axis=0)

# This has been checked using print(), code that has been deleted after check, and every file is read.

In [ ]:
# NOAA token
Token = "UyzWJXDqukWjTLQeXjiHAxHkgloCIgNB" 
# Request New York LaGuardia’s weather station for 2022 
r = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00014732&startdate=2022-01-01&enddate=2022-12-31",headers={"token":Token})
# Sort as json format
weatherJson = json.loads(r.text) 

In [ ]:
# Wrangle data from weather

avg_temps = [item for item in weatherJson["results"] if item["datatype"]=="TAVG"]
date_temp = [item["date"] for item in avg_temps]
temps = [item["value"] for item in avg_temps]

# turn into df, fix date and tmep
df_temps = pd.DataFrame({"date":date_temp, "avgTemp":temps})
df_temps["date"] = [datetime.datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in date_temp] 
df_temps["avgTemp"] = [float(v)/10.0 for v in temps]

df_temps.to_csv("NY_LaGuardia_weather-station_2022")

In [ ]:
# convert citi_df date format to be the same as df_temps
citi_df["date"] = citi_df["started_at"].apply(lambda x: str(x)[0:10]) 
citi_df["date"] = citi_df["date"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d")) 
# drop unneeded columns
citi_df = citi_df.drop(["started_at","ended_at"], axis = 1)

In [ ]:
citi_df.sort_values(by="date") # check, shows than it has none 2021 values

In [ ]:
# drop date out of bounds (includes days from 2021)

# sort first to make faster?
citi_df = citi_df.sort_values(by="date")

import datetime 

start = datetime.datetime.strptime("2022-01-01", "%Y-%m-%d")
end = datetime.datetime.strptime("2022-12-31", "%Y-%m-%d")

to_drop = []

for index, row in citi_df.iterrows():
    if not (start <= row["date"] <= end):
        # print(row["date"])
        to_drop.append(index)

In [ ]:
citi_df = citi_df.drop(to_drop, axis=0)
citi_df = citi_df.reset_index()

In [ ]:
citi_df.sort_values(by="date") # check

In [ ]:
df_merged = citi_df.merge(df_temps, how = "left", on = "date", indicator = True)
df_merged

In [ ]:
df_merged.sort_values(by="date") # check

In [ ]:
df_merged['_merge'].value_counts(dropna = False)

In [ ]:
df_merged.to_csv("Citi_Bike_LG_merge")